# Hyperparameter Tuning

_Summarized by QH_  
_First version: 2023-07-08_  
_Last updated on : 2023-07-08_  

## Model hyperparameters vs model parameters?
* Model hyperparameters are parameters that control the modeling or learning process. 
* Model parameters are the parameters that are estimated through training process.
* Model hyperparameters will set the direction in the training process and will impact the estimation of model parameters.

Think about the knobs on the radios you can use to tune the frequency and volumn - they are the hyperparameters!


## Hyperparameter Importance

Some hyperparameters are more important in terms of determining the model performance than others. 

For example, random forest classifier:
* `n_jobs`: The number of jobs to run in parallel.
* `random_state`: Controls both the randomness of the bootstrapping of the samples used when building trees and and the sampling of the features to consider when looking for the best split at each node.
* `verbose`: Controls the verbosity when fitting and predicting.

do not impact the model performance compared to the following:
* `n_estimators`: The number of trees to build.
* `max_features`: The number of features to consider when looking for best splits.
* `max_depth`: The maximum depth of the tree.
* `min_sample_leaf`: The minimum number of samples required to be at a leaf node.

## How to choose hyperparameters?
The process is called the hyperparameter tuning - we want to tune as best as we can on the hyperparameters that has the lowest predicting error.

### Constraints among hyperparameters
Be careful on the potential conflict of hyperparameters. For example, `LogisticRegression()` has conflicting hyperparameter options of `solver` and `penalty`.
### Avoiding "Silly" choices
Certain values that will definitely not contribute to decent model performance, then avoid them.
* Random Forest
    * low number of trees (`n_estimators`)
* K-Nearest Neighbor
    * <= 2 Neighbors (`n_neighbors`)
* Increasing a hyperparameter by a very small amount compared with its range.

### Grid Search
For each of the hyperparameters you want to tune, list all chosen values and test each combinations and find the combination with the best model performance. For example, Gradient boosting, we want to tune the following hyperparameters:
* `learn_rate`: [0.001, 0.01, 0.1, 0.2]
* `max_depth`: [4, 6, 8, 10, 12, 15, 20, 25, 30]
* `subsample`: [0.4, 0.6, 0.7, 0.8, 0.9]
* `max_features`: ['auto', 'sqrt']

We will test all $4 \times 9 \times 5 \times 2 = 360$ combinations. If we want to use 10 fold cross-validation, then we will in total make $360 \times 10 = 3600$ models.

* Advantanges of Grid Search
    * You are guaranteed to the find the best results in this grid - since you have performed an exhaustive search.
* Disadvantages of Grid Search
    * It is computationally expensive. The cost increases exponentially when adding more hyperparameters and testing more hyperparameter values.
    * It is uninformed. Running models are independent - previous model do not inform the next choice.

You can do mannually as looping through each combination as follows:

In [ ]:
# python packages
import pandas as pd
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets import make_hastie_10_2
from sklearn.metrics import roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# import the data
X, y = make_hastie_10_2(random_state=0)
X_train, X_test = X[:2000], X[2000:]
y_train, y_test = y[:2000], y[2000:]

In [ ]:
# Adjust the list of values to test
learn_rate_list = [0.001, 0.01, 0.1, 0.2, 0.3, 0.5]
max_depth_list = [4,6,8, 10, 12, 15, 20, 25, 30]
subsample_list = [0.7]
max_features_list = ['sqrt']

def gbm_grid_search(learn_rate, max_depth,subsample,max_features):
    model = GradientBoostingClassifier(
    learning_rate=learn_rate,
    max_depth=max_depth,
    subsample=subsample,
    max_features=max_features)
    predictions = model.fit(X_train, y_train).predict(X_test)
    return([learn_rate, max_depth, subsample, max_features, roc_auc_score(y_test, predictions)])

results_list = []
for learn_rate in learn_rate_list:
    for max_depth in max_depth_list:
        for subsample in subsample_list:
            for max_features in max_features_list:
                results_list.append(gbm_grid_search(learn_rate,max_depth, subsample,max_features))
results_df = pd.DataFrame(results_list, columns=['learning_rate', 'max_depth', 'subsample', 'max_features','auc'])
print(results_df.head())

In [ ]:
# evaluate the performance by hyperparameter values
sns.scatterplot(data=results_df, x='max_depth', y='auc')
plt.show()

Or you can use `GridSearchCV` in `scikit-learn` as follows:

```
sklearn.model_selection.GridSearchCV(
    estimator,
    param_grid, 
    scoring=None, 
    n_jobs=None, # Number of jobs to run in parallel.
    refit=True, # Refit an estimator using the best found parameters on the whole dataset.
    cv=10,
    verbose=0, # Controls the verbosity: the higher, the more messages. 
    pre_dispatch='2*n_jobs', # Controls the number of jobs that get dispatched during parallel execution.
    error_score='raise', # Value to assign to the score if an error occurs in estimator fitting.
    return_train_score=True # Computing training scores is used to get insights on how different parameter settings impact the overfitting/underfitting trade-off.
)
```

1. Define the method and only one method (e.g. Random Forest) we use for modeling - `estimator`
2. Choose and hyperparameters and Define the hyperparameter value grid as a dictionary - `param_grid`
3. Set a cross-validation scheme as how many folds to be performed - `cv`
4. Choose/Define a scoring function to evelation model performance - `scoring`
5. Choose more information for the Grid Search Process.



In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {
    'learning_rate': [0.001, 0.01, 0.1, 0.2, 0.3, 0.5],
    'max_depth': [4,6,8, 10, 12, 15, 20, 25, 30],
    'subsample': [0.7],
    'max_features': ['sqrt']
}

grid_gb_clf = GridSearchCV(
    GradientBoostingClassifier(),
    param_grid=param_grid, 
    scoring='roc_auc', 
    n_jobs=4, # Number of jobs to run in parallel.
    refit=True, # Refit an estimator using the best found parameters on the whole dataset.
    cv=5,
    verbose=0, # Controls the verbosity: the higher, the more messages. 
    pre_dispatch='2*n_jobs', # Controls the number of jobs that get dispatched during parallel execution.
    error_score='raise', # Value to assign to the score if an error occurs in estimator fitting.
    return_train_score=True # Computing training scores is used to get insights on how different parameter settings impact the overfitting/underfitting trade-off.
)
# Fit the training data
grid_gb_clf.fit(X_train, y_train)

# Get the training results
grid_gb_clf.cv_results_

### Randomized Search
Similar to Grid Search, we define the hypermeter value grid. The difference is that we don't search the entire grid but randomly choose a range for each parameter to test.

Bengio & Bergstra (2012): This paper shows empirically and theoretically that randomly chosen trials are more efficient for hyper-parameter optimization than trials on a grid. 
There are two main reasons:
1. Not all hyperparameters are of equal importantance to contribute to model performance
2. As long as we try enough time, we can achieve a high probabilty of getting a decent good model - magic of probability!

Imagine for 100 combinations of hyperparameters, there are 5 of them that can generage best model performance.
* If we try 1 time, the probability of missing the best selections is $1-0.05 = 0.95$
* If we try 2 times, the probability of missing the best selections is $(1-0.05) \times (1-0.05) = 0.95^2$
* ...
* If we try n times, the probability of missing the best selections is $(1-0.05) \times \cdots \times (1-0.05) = 0.95^n$ which means the if we try n times, the probability of hitting the best selections is $1 - 0.95^n$ - We only need 59 times of trial and the probability of selecting the best 5 is greater than 95%!

You can use `RandomizedSearchCV` in `scikit-learn` as follows:

```
sklearn.model_selection.RandomizedSearchCV(
    estimator,
    param_distributions,
    n_iter, 
    scoring=None, 
    n_jobs=None, # Number of jobs to run in parallel.
    refit=True, # Refit an estimator using the best found parameters on the whole dataset.
    cv=10,
    verbose=0, # Controls the verbosity: the higher, the more messages. 
    pre_dispatch='2*n_jobs', # Controls the number of jobs that get dispatched during parallel execution.
    error_score='raise', # Value to assign to the score if an error occurs in estimator fitting.
    return_train_score=True # Computing training scores is used to get insights on how different parameter settings impact the overfitting/underfitting trade-off.
)
```

1. Define the method and only one method (e.g. Random Forest) we use for modeling - `estimator`
2. Choose and hyperparameters and Define the hyperparameter value distribution as a dictionary - `param_distributions`
3. Decide how many combinations you want to try - `n_iter` 
4. Set a cross-validation scheme as how many folds to be performed - `cv`
5. Choose/Define a scoring function to evelation model performance - `scoring`
6. Choose more information for the Grid Search Process.

## References
1. [Hyperparameter Tuning Course on Datacamp](https://app.datacamp.com/learn/courses/hyperparameter-tuning-in-python)